In [1]:
import os
import natsort
from tqdm.notebook import tqdm, trange
import sys
import argparse
import re
import bioread
import numpy as np
import pandas as pd
from scipy import io as sio
import pdb

In [2]:
def SelectAcqFiles(file_list):
    # file_list 중, 'acq' 문자열을 포함한 것만 남김
    # file_list : 검사하고자 하는 file 이름이 담긴 list
    print("\nTest... if acq or not")
    result = []
    for i in range(len(file_list)):
        test = file_list[i][-3:]
        if test == 'acq':
            result.append(file_list[i])
    print('Filtered acq files: {}'.format(result))
    return result

def clean(s):
    '''Take a string and construct a valid variable name.
    Used to ensure that channel names are able to be used in MATLAB.
    Adapted from https://stackoverflow.com/questions/3303312/how-do-i-convert-a-string-to-a-valid-variable-name-in-python'''

    s = s.lower().strip() # preliminary cleaning
    s = re.sub('[^0-9a-zA-Z_]', '', s) # include only valid characters
    s = re.sub('^[^a-zA-Z]+', '', s) # first character must be a letter

    return s

def parse_data(data):
    '''Read in ACQ file using njvack's bioread package (https://github.com/uwmadison-chm/bioread)'''
    d = {} # new dictionary to be saved with scipy.io

    # Add channel data
    iter = 1
    for channel in data.channels:
        chan_name = channel.name.lower().strip().replace(' ', '_')
        d[clean(channel.name+'_'+str(iter))] = {
            'wave':channel.data,
            'Fs': channel.samples_per_second,
            'unit': channel.units,
        }
        iter = iter + 1

    return d


## 분석할 subject 선택

In [3]:
# subject별 데이터가 저장된 경로
data_path = 'E:\\B2X\\2차년도\\03_pilot\\subject_data'

# subject_data/01_KDH
# subject_data/02_HHB
# subject_data/...

# subject list
def OnlyFolder(path):
    # return only folder, not file
    
    dir_list = os.listdir(path) # get all directory below the path
    output = []
    for i in range(len(dir_list)):
        temp_path = os.path.join(path, dir_list[i])
        if os.path.isdir(temp_path) == True:
            output.append(dir_list[i])
    return output

total_subject_list = OnlyFolder(data_path)
total_subject_list = natsort.natsorted(total_subject_list) # 번호 순서대로 정렬
print('Total Subjects: {}'.format(total_subject_list))

choose_sub = range(len(total_subject_list))
# choose_sub = [0,1]
sub_list = []
for sub_num in range(len(choose_sub)):    
    sub_list.append(total_subject_list[choose_sub[sub_num]])
    
print('\nTarget Subjects: {}'.format(sub_list))


Total Subjects: ['01_KDH', '02_HHB', '03_PSR', '04_CJK', '05_CKM', '06_KDH', '07_PSR', '08_HHB', '09_CKM', '10_CSJ', '11_KMS', '12_CJK', '13_CSJ', '14_KMS', '15_LJH', '16_PBJ', '17_PBJ', '18_SYH', '19_SYH', '20_LJH']

Target Subjects: ['01_KDH', '02_HHB', '03_PSR', '04_CJK', '05_CKM', '06_KDH', '07_PSR', '08_HHB', '09_CKM', '10_CSJ', '11_KMS', '12_CJK', '13_CSJ', '14_KMS', '15_LJH', '16_PBJ', '17_PBJ', '18_SYH', '19_SYH', '20_LJH']


In [4]:
save_dir_all_sub = 'E:\\B2X\\2차년도\\03_pilot\\biopac_mat'

for sub_num in tqdm(range(len(choose_sub)), desc='Total Processing'):
    print('------------------------------------')
    print(sub_list[sub_num])
    
    # mat 파일을 저장할 subject 경로
    save_dir_each_sub = os.path.join(save_dir_all_sub, sub_list[sub_num])
    if not os.path.exists(save_dir_each_sub):
        os.makedirs(save_dir_each_sub)
    
    
    # 현재 sub_num에 해당하는 subject의 acq 파일이 존재하는 경로
    subject_acq_path = data_path + '\\' + sub_list[sub_num] + '\\Biopac'
    
    # acq 경로에서, 확장자명이 acq인 파일들만 선택
    acq_list = SelectAcqFiles(os.listdir(subject_acq_path))
    print('\nData Extraction Start ...')
    
    for acq_num in range(len(acq_list)):
        print(acq_list[acq_num], end='  ...')
        data = bioread.read_file(subject_acq_path+'\\'+acq_list[acq_num])
        d_list = parse_data(data)
        save_path = os.path.join(save_dir_each_sub, acq_list[acq_num].replace('.acq', '.mat'))
        sio.savemat(save_path, d_list, oned_as='column', do_compression=True)
        print('OK !')

    
    

Total Processing:   0%|          | 0/20 [00:00<?, ?it/s]

------------------------------------
01_KDH

Test... if acq or not
Filtered acq files: ['Stim_S0.acq', 'Stim_S1_S100.acq', 'Stim_S2_S100.acq', 'Stim_S3_N100.acq', 'Stim_S4_S020.acq']

Data Extraction Start ...
Stim_S0.acq  ...OK !
Stim_S1_S100.acq  ...OK !
Stim_S2_S100.acq  ...OK !
Stim_S3_N100.acq  ...OK !
Stim_S4_S020.acq  ...OK !
------------------------------------
02_HHB

Test... if acq or not
Filtered acq files: ['Stim_S0.acq', 'Stim_S1_S020.acq', 'Stim_S2_N100.acq', 'Stim_S3_S100.acq', 'Stim_S4_N020.acq']

Data Extraction Start ...
Stim_S0.acq  ...OK !
Stim_S1_S020.acq  ...OK !
Stim_S2_N100.acq  ...OK !
Stim_S3_S100.acq  ...OK !
Stim_S4_N020.acq  ...OK !
------------------------------------
03_PSR

Test... if acq or not
Filtered acq files: ['Stim_S0.acq', 'Stim_S1_N020.acq', 'Stim_S2_S020.acq', 'Stim_S3_N100.acq', 'Stim_S4_S100.acq']

Data Extraction Start ...
Stim_S0.acq  ...OK !
Stim_S1_N020.acq  ...OK !
Stim_S2_S020.acq  ...OK !
Stim_S3_N100.acq  ...OK !
Stim_S4_S100.acq  ...